## Thought process:
- need to train this model as it's going through the maze in order to get previous image and use it to train the model
- prob need a combo of code from Example-FastAI-Marchese and FAI-NN-Marchese (additional code prob using pytorch)
- to get it to acc go through the maze automatically, probably will need navigator from autogen
    - maybe could use sequencial images, but may run into issues with sorting images (images not being all in the same folders)
    - but with using navigator from autogen, there's still that issue bc that script actually doesn't return directions. It does, however, directly put moves/photos into the proper folders (w/ their classfication), meaning there is a way to know what each move is as well as the model's previous move.
- another possibility is to try this on a model I've already trained in the maze...prob simpler than changing autogen

## Plan
- get data in (thinking about creating function to go through move_list and add prev_move into tuple so I don't have to worry about the ordering of the input data when it goes into the model)
- split data (train/validate) (maybe acc split list to do this???)
- train model with image data, using move_list to get prev_image

In [2]:
%matplotlib notebook
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

import sys
from fastai.vision.all import *

sys.path.append("../PycastWorld")
sys.path.append("../Gym")

from gym_pycastworld.PycastWorldEnv import PycastWorldEnv
from autogen_edited_Marchese import Navigator

In [3]:
maze = "../Worlds/maze.txt"
show_freq = 0
img_dir = "../Notebooks/data"
move_list = list()

navigator = Navigator(maze, img_dir)

j = 0
while j < navigator.num_directions - 1:
    move_list.extend(navigator.navigate(j, show_dir=True, show_freq=show_freq))                 
    j += 1

print(move_list)

Directions: 1, 3, Dir.NORTH
Directions: 1, 5, Dir.NORTH
Directions: 1, 7, Dir.NORTH
Directions: 1, 9, Dir.EAST
Directions: 3, 9, Dir.NORTH
Directions: 3, 11, Dir.NORTH
Directions: 3, 13, Dir.NORTH
Directions: 3, 15, Dir.WEST
Directions: 1, 15, Dir.NORTH
Directions: 1, 17, Dir.EAST
Directions: 3, 17, Dir.EAST
Directions: 5, 17, Dir.SOUTH
Directions: 5, 15, Dir.EAST
Directions: 7, 15, Dir.EAST
Directions: 9, 15, Dir.NORTH
Directions: 9, 17, Dir.NORTH
Directions: 9, 19, Dir.EAST
Directions: 11, 19, Dir.EAST
Directions: 13, 19, Dir.EAST
Directions: 15, 19, Dir.EAST
Directions: 17, 19, Dir.EAST
Directions: 19, 19, Dir.EAST
[['straight', '00001.png'], ['straight', '00002.png'], ['straight', '00003.png'], ['straight', '00004.png'], ['straight', '00005.png'], ['straight', '00006.png'], ['straight', '00007.png'], ['straight', '00008.png'], ['straight', '00009.png'], ['straight', '00010.png'], ['straight', '00011.png'], ['straight', '00012.png'], ['straight', '00013.png'], ['straight', '00014.pn

In [4]:
# Add previous move into the tuples in move_list
for i in range(len(move_list)):
    if i == 0:
        prev_move = "straight"
    else:
        prev_move = move_list[i-1][0]
    move_list[i].append(prev_move)
move_list

[['straight', '00001.png', 'straight'],
 ['straight', '00002.png', 'straight'],
 ['straight', '00003.png', 'straight'],
 ['straight', '00004.png', 'straight'],
 ['straight', '00005.png', 'straight'],
 ['straight', '00006.png', 'straight'],
 ['straight', '00007.png', 'straight'],
 ['straight', '00008.png', 'straight'],
 ['straight', '00009.png', 'straight'],
 ['straight', '00010.png', 'straight'],
 ['straight', '00011.png', 'straight'],
 ['straight', '00012.png', 'straight'],
 ['straight', '00013.png', 'straight'],
 ['straight', '00014.png', 'straight'],
 ['straight', '00015.png', 'straight'],
 ['straight', '00016.png', 'straight'],
 ['straight', '00017.png', 'straight'],
 ['straight', '00018.png', 'straight'],
 ['straight', '00019.png', 'straight'],
 ['straight', '00020.png', 'straight'],
 ['straight', '00021.png', 'straight'],
 ['straight', '00022.png', 'straight'],
 ['straight', '00023.png', 'straight'],
 ['straight', '00024.png', 'straight'],
 ['straight', '00025.png', 'straight'],


In [5]:
# Making the list of list, a list of tuples
move_list = [tuple(move) for move in move_list]
move_list

[('straight', '00001.png', 'straight'),
 ('straight', '00002.png', 'straight'),
 ('straight', '00003.png', 'straight'),
 ('straight', '00004.png', 'straight'),
 ('straight', '00005.png', 'straight'),
 ('straight', '00006.png', 'straight'),
 ('straight', '00007.png', 'straight'),
 ('straight', '00008.png', 'straight'),
 ('straight', '00009.png', 'straight'),
 ('straight', '00010.png', 'straight'),
 ('straight', '00011.png', 'straight'),
 ('straight', '00012.png', 'straight'),
 ('straight', '00013.png', 'straight'),
 ('straight', '00014.png', 'straight'),
 ('straight', '00015.png', 'straight'),
 ('straight', '00016.png', 'straight'),
 ('straight', '00017.png', 'straight'),
 ('straight', '00018.png', 'straight'),
 ('straight', '00019.png', 'straight'),
 ('straight', '00020.png', 'straight'),
 ('straight', '00021.png', 'straight'),
 ('straight', '00022.png', 'straight'),
 ('straight', '00023.png', 'straight'),
 ('straight', '00024.png', 'straight'),
 ('straight', '00025.png', 'straight'),


In [ ]:
# Specify output categories for NN and path to get training/validation images
move_type = 'straight', 'right', 'left'
path = Path("data")

In [ ]:
# Get images and verify them
fns = get_image_files(path)
failed = verify_images(fns)
failed

In [ ]:
# Split up dataset, load in images
dls = ImageDataLoaders.from_folder(path, valid_pct=0.2, seed=42, item_tfms=Resize(128)) 

Question: Can I use ImageDataLoaders.from_folder here? I don't think I can bc I will probably need to go image by image and i'm not sure if something like looping through dls will give an image at a time...

In [6]:
model = load_learner("Move_NN_750.pkl")
model.model

Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  

In [29]:
models.resnet18(pretrained = True)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [30]:
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.cnn = model
        self.cnn.fc = nn.Linear(self.cnn.fc.in_features, 512)
        
        self.fc2 = nn.Linear(512 + 1, 3)
        
    def forward(self, image, data):
        x1 = self.cnn(image)
        x2 = data
        
        x = torch.cat((x1, x2), dim=1)
        x = F.relu(self.fc2(x))
        return x

In [ ]:
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.cnn = model
        
        self.fc1 = nn.Linear(self.cnn.fc.out_features + 1, 512)
        self.fc2 = nn.Linear(512, 3)
        
    def forward(self, image, data):
        x1 = self.cnn(image)
        x2 = data
        
        x = torch.cat((x1, x2), dim=1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
net = MyModel()

In [ ]:
# defining loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
for epoch in range(4):
    
    for i in range():
        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass: Compute predicted y by passing x to the model
        y_pred = net(image, prev_move)

        # Compute and print loss
        loss = nn.L1Loss()
        loss_val = loss(y_pred, y)
        if i%5==0:
            print(i, loss_val.item())

        # Perform a backward pass, and update the weights.
        loss_val.backward(retain_graph=True)
        optimizer.step()

print("Finished Training")